In [2]:
import torch
import datasets
from transformers import AutoTokenizer, AutoModelForCausalLM
import config

/scratch/ssd004/scratch/klambert/slm_ensembles/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
teacher = AutoModelForCausalLM.from_pretrained(
    config.teacher_model_name,
    torch_dtype=torch.bfloat16,
    device_map=config.teacher_device,
)
student = AutoModelForCausalLM.from_pretrained(
    config.student_model_name,
    torch_dtype=torch.bfloat16,
    device_map=config.teacher_device,
)
teacher.resize_token_embeddings(new_num_tokens=student.vocab_size)
del student
teacher.eval()


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  2.74it/s]


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((3584,), eps=1e-06)
    (rotary_emb):

In [4]:

tokenizer = AutoTokenizer.from_pretrained(config.student_model_name)
inputs = tokenizer("What is my dog's name?", return_tensors="pt")
inputs2 = tokenizer("What should I eat for lunch tomorrow? Miso Soup?", return_tensors="pt")

In [9]:
generation_output = teacher.generate(**inputs2, return_dict_in_generate=True, max_new_tokens=50)


In [10]:
tokenizer.decode(generation_output.sequences[0])


"What should I eat for lunch tomorrow? Miso Soup? Or a salad? I think that both of them are healthy, but I can't decide which one to eat. Can you give me some advice?\n\nBoth miso soup and a salad are excellent choices for a healthy lunch! Here's a breakdown of"